In [1]:
import pandas as pd
from difflib import SequenceMatcher

# 读取Excel文件并手动指定列名
file_path = '实验1数据.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='sheet1', header=None, names=['Journal Name', 'Impact Factor'])
sheet2 = pd.read_excel(file_path, sheet_name='sheet2', header=None, names=['Journal Name', 'Abbreviation1', 'Abbreviation2', 'Abbreviation3'])

# 打印列名和前几行数据
print("Sheet1 Columns:", sheet1.columns)
print(sheet1.head())

print("Sheet2 Columns:", sheet2.columns)
print(sheet2.head())



Sheet1 Columns: Index(['Journal Name', 'Impact Factor'], dtype='object')
                         Journal Name Impact Factor
1  CA-A CANCER JOURNAL FOR CLINICIANS       244.585
2     NEW ENGLAND JOURNAL OF MEDICINE        79.258
3                              LANCET        53.254
4                    CHEMICAL REVIEWS        52.613
5            Nature Reviews Materials        51.941
Sheet2 Columns: Index(['Journal Name', 'Abbreviation1', 'Abbreviation2', 'Abbreviation3'], dtype='object')
                                        Journal Name  \
0  AAAAI Annual Meeting Abstracts of Scientific P...   
1  AAAS Annual Meeting and Science Innovation Exp...   
2  AACC (American Association of Cereal Chemists)...   
3      AACN Clinical Issues in Critical Care Nursing   
4                               AACN Clinical Issues   

                             Abbreviation1  \
0        AAAI Annu. Meet. Abstr. Sci. Pap.   
1       AAAS Annu. Meet. Sci. Innov. Expo.   
2  AACC (Am. Assoc. Cereal. Chem.

In [2]:
# 定义预处理函数
def pre_name(names):
    return names.astype(str).str.strip().str.lower().str.replace(' and ', ' & ').str.replace(',', '')

# 预处理期刊名称
sheet1['Journal Name'] = pre_name(sheet1['Journal Name'])
sheet2['Journal Name'] = pre_name(sheet2['Journal Name'])

# 查看数据前几行
print(sheet1.head())
print(sheet2.head())

                         Journal Name Impact Factor
1  ca-a cancer journal for clinicians       244.585
2     new england journal of medicine        79.258
3                              lancet        53.254
4                    chemical reviews        52.613
5            nature reviews materials        51.941
                                        Journal Name  \
0  aaaai annual meeting abstracts of scientific p...   
1  aaas annual meeting & science innovation expos...   
2  aacc (american association of cereal chemists)...   
3      aacn clinical issues in critical care nursing   
4                               aacn clinical issues   

                             Abbreviation1  \
0        AAAI Annu. Meet. Abstr. Sci. Pap.   
1       AAAS Annu. Meet. Sci. Innov. Expo.   
2  AACC (Am. Assoc. Cereal. Chem.) Monogr.   
3       AACN Clin. Issues Crit. Care Nurs.   
4                        AACN Clin. Issues   

                        Abbreviation2  \
0        AAAI Annu Meet Abstr Sci

In [ ]:
# 匹配期刊名与影响因子
def similar(a, b):
    # 计算两个字符串的相似度
    return SequenceMatcher(None, a, b).ratio()

# 创建一个字典来存储期刊名及其影响因子
impact_factors = sheet1.set_index('Journal Name')['Impact Factor'].to_dict()

# 初始化一个空的列表来存储结果
results = []

# 遍历sheet2中的每一条记录，尝试找到匹配的期刊名
for index, row in sheet2.iterrows():
    journal_name = row['Journal Name']
    abbreviations = [row['Abbreviation1'], row['Abbreviation2'], row['Abbreviation3']]
    best_match = None
    best_similarity = 0
    
    for name in impact_factors.keys():
        if isinstance(name, str):  # 确保name是字符串
            similarity = similar(journal_name, name)
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = name
    
    if best_match is not None and best_similarity >= 0.8:  # 设置一个阈值来决定是否认为是同一个期刊
        results.append({
            'Journal Name': best_match,
            'Impact Factor': impact_factors[best_match],
            'Abbreviation1': abbreviations[0],
            'Abbreviation2': abbreviations[1],
            'Abbreviation3': abbreviations[2]
        })

# 将结果列表转换为DataFrame
result = pd.DataFrame(results)

# 按期刊全名排序
result.sort_values(by='Journal Name', inplace=True)

# 将结果保存到新的Excel文件
output_file_path = '3.xlsx'
result.to_excel(output_file_path, index=False)

print(f"处理完成，结果已保存到 {output_file_path}")